This notebook provides an example of developing a quality control (QC) method for use with the KPF DRP.

In [1]:
# Import packages for reading files
from modules.Utils.kpf_parse import get_datecode
from kpfpipe.models.level0 import KPF0

In [2]:
def L0_data_products_check(L0, data_products=['auto'], debug=False):
    """
    This Quality Control function checks if the specified data_products 
    in an L0 file are present and if their data extensions are populated 
    with arrays of non-zero size.
    
    Args:
         L0 - an L0 object
         data_products - L0 data_products to check (list)
                         possible elements = 'auto', 'all', 
                                             'Green', 'Red', 'CaHK', 'ExpMeter', 
                                             'Guider', 'Telemetry', 'Pyrheliometer'
                                             (note that 'all' should be used rarely since good data
                                              could be missing some extensions, e.g. CaHK, Pyrheliometer)
         debug - an optional flag.  If True, missing data products are noted.

     Returns:
         QC_pass - a boolean signifying that the QC passed for failed
    """
    
    # determine which extensions should be in the L0 file
    if data_products == ['auto']:
        #data_products = []
        # first add triggrered cameras (Green, Red, CaHK, ExpMeter)
        trigtarg = L0.header['PRIMARY']['TRIGTARG']
        print(trigtarg)
        print(data_products)
        if len(trigtarg) > 0:
            data_products = trigtarg.split(',')
        # add Guider
        if hasattr(L0, 'GUIDER_AVG'):
            data_products.append('Guider')
        if hasattr(L0, 'guider_avg'):  # some early files had lower case
            data_products.append('Guider')
        # add Telemetry
        print(data_products)
        if hasattr(L0, 'TELEMETRY'):
            data_products.append('Telemetry')
        # add Pyrheliometer
        if hasattr(L0, 'SOCAL PYRHELIOMETER'):
            data_products.append('Pyrheliometer')
        if debug:
             print('Data products that are supposed to be in this L0 file: ' + str(data_products))
 
    QC_pass = True
    
    # Use helper funtion to get data products and check their characteristics.
    from modules.Utils.kpf_parse import get_data_products_L0
    data_products_present = get_data_products_L0(L0)
    if debug:
        print('Data products in L0 file: ' + str(data_products_present))

    # Check for specific data products
    possible_data_products = ['Green', 'Red', 'CaHK', 'ExpMeter', 'Guider', 'Telemetry', 'Pyrheliometer']
    for dp in possible_data_products:
        if (dp in data_products) or ('all' in data_products):
            if not dp in data_products_present:
                QC_pass = False
                if debug:
                    print(dp + ' not present in L0 file. QC(L0_data_products_check) failed.')
    
    return QC_pass

In [3]:
# Use the from_fits method from the KPF DRP (not astropy.fits) to read in L0 files
ObsID = 'KP.20230829.76026.81' # good KPF file
L0_filename = '/data/L0/' + get_datecode(ObsID) + '/' + ObsID + '.fits'
L0a = KPF0.from_fits(L0_filename)

ObsID = 'KP.20231108.77769.16'  # file with missing Red
L0_filename = '/data/L0/' + get_datecode(ObsID) + '/' + ObsID + '.fits'
L0b = KPF0.from_fits(L0_filename)

In [4]:
# Run the QC on an L0 file that does not have a 'Red' extension.  It will fail.
L0_data_products_check(L0b, data_products=['Green', 'Red'], debug=True)

Data products in L0 file: ['Green', 'ExpMeter', 'Telemetry', 'Pyrheliometer']
Red not present in L0 file. QC(L0_data_products_check) failed.


False

In [5]:
# Run the QC on an L0 file to determine if has non-zero data arrays.  It will fail.
L0_data_products_check(L0b, data_products=['auto'], debug=True)

Green,Red,ExpMeter
['auto']
['Green', 'Red', 'ExpMeter']
Data products that are supposed to be in this L0 file: ['Green', 'Red', 'ExpMeter', 'Telemetry', 'Pyrheliometer']
Data products in L0 file: ['Green', 'ExpMeter', 'Telemetry', 'Pyrheliometer']
Red not present in L0 file. QC(L0_data_products_check) failed.


False

In [6]:
# Run the QC on an L0 file that does not have the 'CaHK' extension.  It will fail.
L0_data_products_check(L0b, data_products=['CaHK'], debug=True)

Data products in L0 file: ['Green', 'ExpMeter', 'Telemetry', 'Pyrheliometer']
CaHK not present in L0 file. QC(L0_data_products_check) failed.


False

In [7]:
# Run the QC on aa good L0 file.  It will pass.
L0_data_products_check(L0a, data_products=['auto'], debug=True)

Green,Red,ExpMeter
['auto']
['Green', 'Red', 'ExpMeter']
Data products that are supposed to be in this L0 file: ['Green', 'Red', 'ExpMeter', 'Telemetry', 'Pyrheliometer']
Data products in L0 file: ['Green', 'Red', 'ExpMeter', 'Telemetry', 'Pyrheliometer']


True